In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # normalize to range [-1,1]
])

trainset = datasets.MNIST('~/dataset/MNIST_data/', download=True, train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.ReLU(),
            nn.Linear(12, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()  # to force the output to be between 0 and 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
def train(model, num_epochs, data_loader, criterion, optimizer, device):
    model.train()
    for epoch in range(num_epochs):
        for data in data_loader:
            img, _ = data
            img = img.view(img.size(0), -1)
            noisy_img = img + 0.5 * torch.randn(*img.shape)  # add some noise to the images
            noisy_img = np.clip(noisy_img, 0., 1.)
            noisy_img = noisy_img.to(device)

            output = model(noisy_img)
            loss = criterion(output, img.to(device))  # compute the loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'epoch [{epoch+1}/{num_epochs}], loss:{loss.item():.4f}')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_data_loader = ...  # define your data loader here
num_epochs = 100  # define your number of epochs

train(model, num_epochs, train_data_loader, criterion, optimizer, device)

In [2]:
torch.cuda.is_available() 

True